# Creating a variational classifier with PennyLane

In the guest lecture on quantum kernels, we saw in Section 4 that feature embeddings can be used in combination with variational classifiers that analyse data in "quantum feature space". This notebook shows a simplified implementation of the idea in the PennyLane framework for hybrid optimization. 

Since training variational quantum circuits can be time consuming, we show what decision boundaries for a simple classification task a quantum classifier gives rise to.

### The variational circuit

The circuit we discussed contains the following elements:
1. a quantum circuit $S_x$ that depends on an input $x$, which maps the input to a quantum state,
2. a variational circuit $U_{\theta}$, which depends on parameters $\theta$ that can be optimized,
3. a computational basis measurement of the first qubit. 

The routine can be interpreted as a classifier that takes inputs $x$ and maps them to outputs $y = \langle \hat{\sigma}_z\rangle$. The classifier is "trained" by finding parameters $\theta$ given data of input-output samples.

<img src="figures/variational_classifier.png" alt="circuit" style="width:400px;"/>

For this tutorial to run, you need to install the PennyLane library. In most cases this can be done with 

    pip install pennylane 

**An extended example and more details can be found at *pennylane.ai*.**

### Importing libraries

Let us import some helpers from scikit-learn:

In [3]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

KeyboardInterrupt: 

If you installed the PennyLane library correctly, you can run the following imports.

In [4]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer

We also need matplotlib for visualisation. 

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline  

RuntimeError: can't start new thread

### Implementing the circuit

As a quantum device we use PennyLane's default qubit simulator with 2 qubits. We can create the quantum device as follows. 

*Note 1: using another backend than `default.qubit`, we can also run the circuit on a real quantum device through a cloud service.*

*Note 2: the keyword `wires` stands for the number of qubits. Since PennyLane can also handle non-qubit (i.e., continuous-variable systems), wires generally refers to the number of subsystems.*

In [ ]:
dev = qml.device('default.qubit', wires=2)

We now implement the two quantum circuits in PennyLane, using rather arbitrary gate sequences (find better ones!). Let is first define a feature embedding (or state preparation) circuit $S_x$...

In [ ]:
def S(x): 
    qml.RX(x[0], 0)
    qml.RX(x[1], 1)

... and a classification circuit $U(\theta)$.

In [ ]:
def U(theta):  
    qml.Rot(theta[0], theta[1], theta[2], 0)
    qml.Rot(theta[3], theta[4], theta[5], 0)
    qml.CNOT([1, 0])

We call these two routines in a quantum classifier function. In fact, $U(\theta)$ is called several times, depending on how many lists of parameters `thetas` contains. After this, the expectation of the Pauli Z operator is measured with respect to the final state. 

The line "@qml.qnode(dev)" is a *decorator* that turns the function into a QNode object, PennyLane's representation of quantum computations. Note that in `qclassifier` no classical processing (e.g., addition, multiplication, etc) is allowed, but only quantum information processing (gates and expectation values). You can find more information on https://pennylane.readthedocs.io/en/latest/code/qnode.html .

In [ ]:
num_layers = 3

@qml.qnode(dev)
def qclassifier(thetas, x=None):
    S(x)
    for theta in thetas:
        U(theta)
    return qml.expval.PauliZ(0)

For optimization we need a cost function. The cost compares targets Y and model outputs with a square loss function. PennyLane needs to be able to "pass gradients" through these functions, which is ensured by previously importing PennyLane's native numpy library version as `np`. (This import makes most numpy operations accessible to automatic differentiation; in other words, PennyLane can compute gradients of functions constructed from QNodes and numpy operations).

In [ ]:
def cost(theta, X, Y):
    Y_ = np.array([qclassifier(theta, x=x) for x in X])
    loss = np.mean(np.abs(Y - Y_)**2)
    return loss 

### Loading data

We generate a simple dataset of two features. To simplify things we will not split it into training and validation sets but just try to fit the data.

In [ ]:
X, y = make_classification(n_samples=30, n_features=2, n_informative=2, n_redundant=0, 
                           n_repeated=0, n_classes=2, random_state=6, class_sep=1.4)

We can visualise the data with matplotlib.

In [ ]:
plt.figure()
plt.scatter(X[:, 0][y==0], X[:, 1][y==0], c='r', marker='^', edgecolors='k', label="train 0")
plt.scatter(X[:, 0][y==1], X[:, 1][y==1], c='b', marker='^', edgecolors='k', label="train 1")
plt.legend()
plt.show()

### Visualising the decision boundary

Let's have a look at the decision boundaries that the quantum classifier "naturally" gives rise to if it is not trained.

You can change the parameter `num_layers` above and run its cell again to play around with the shape of the decision boundary. (The parameter is defined in the same cell as the `qclassifier` in order to make sure you redefine the classifier as well once `num_layers` changes.)

*PS: The plot takes some seconds to be built.*

In [ ]:
np.random.seed(10)
thetas = 3*np.random.rand(num_layers, 6)

In [ ]:
# make data grid for contour plot
xx, yy = np.meshgrid(np.linspace(-2.5, 2.5, 20), np.linspace(-2.5, 2.5, 20))
X_grid = [np.array([x, y]) for x, y in zip(xx.flatten(), yy.flatten())]

# plot decision regions
plt.figure()
cm = plt.cm.RdBu
predictions_grid = [qclassifier(thetas, x=x) for x in X_grid]
Z = np.reshape(predictions_grid, xx.shape)
cnt = plt.contourf(xx, yy, Z, levels=np.arange(-1, 1.1, 0.2), cmap=cm, alpha=.8)
plt.colorbar(cnt, ticks=[0, 0.5, 1])

# plot data
plt.scatter(X[:, 0][y==0], X[:, 1][y==0], c='r', marker='^', edgecolors='k', label="train 0")
plt.scatter(X[:, 0][y==1], X[:, 1][y==1], c='b', marker='^', edgecolors='k', label="train 1")

plt.ylim(-2.5, 2.5)
plt.xlim(-2.5, 2.5)
plt.show()

# Training

The model can be trained by updating the parameters with stochastic gradient descent. In each step, a fair bit of quantum computation is simulated in the background to get gradients for each data point, and training takes some time. 

In [ ]:
o = GradientDescentOptimizer(0.01)

for it in range(40):
    
    thetas = o.step(lambda v: cost(v, X, y), thetas)
    print("Cost in step" , it+1 , ":", cost(thetas, X, y))

Finally, we can evaulate the accuracy on the training set, comparing model predictions with the target labels `y`. If the accuracy is `1.0`, all data points have been classified correctly by the model. Of course, this does not tell us anything about the model's generalisation performance.

*You can run the cell above again to train for another 40 steps.*

In [ ]:
pred = [1 if qclassifier(thetas, x=x_) > 0 else 0 for x_ in X]
accuracy_score(pred, y)

Let's plot the new decision regions.

In [ ]:
# make data for plot
xx, yy = np.meshgrid(np.linspace(-2.5, 2.5, 20), np.linspace(-2.5, 2.5, 20))
X_grid = [np.array([x, y]) for x, y in zip(xx.flatten(), yy.flatten())]

# start plot
plt.figure()
cm = plt.cm.RdBu

# plot decision regions
predictions_grid = [qclassifier(thetas, x=x) for x in X_grid]
Z = np.reshape(predictions_grid, xx.shape)
cnt = plt.contourf(xx, yy, Z, levels=np.arange(-1, 1.1, 0.2), cmap=cm, alpha=.8)
plt.colorbar(cnt, ticks=[0, 0.5, 1])

# plot data
plt.scatter(X[:, 0][y==0], X[:, 1][y==0], c='r', marker='^', edgecolors='k', label="train 0")
plt.scatter(X[:, 0][y==1], X[:, 1][y==1], c='b', marker='^', edgecolors='k', label="train 1")

plt.ylim(-2.5, 2.5)
plt.xlim(-2.5, 2.5)
plt.show()